# Fortran Style System — Global style configuration for code generation

This notebook demonstrates the **global Fortran style system** in macrofor, which allows you to set the Fortran format (F77 or F90) once at the beginning of your script. All subsequent code generation will automatically use that style.

Workflow:
1. Set the global Fortran style using `set_fortran_style('f77')` or `set_fortran_style('f90')`.
2. All macrofor functions (`commentf`, `subroutinef`, `genfor`, etc.) automatically adapt to the current style.
3. F77 style uses `c` comments, 72-character lines, and fixed format. F90 style uses `!` comments, 132-character lines, and free format.

In [9]:
import sys
sys.path.insert(0, '..')

from macrofor.api import (
    set_fortran_style, get_fortran_style,
    genfor, programm, commentf, subroutinef, subroutinem,
    declaref, equalf, dom
)

## 1. Fortran 77 Style (Strict)

Set the style to F77 - all subsequent code will use:
- `c` for comments (column 1)
- 72 character line limit
- Fixed format continuation

In [10]:
# Set global style to F77
set_fortran_style('f77')

# Check current style
style = get_fortran_style()
print(f"Format: {style.format}")
print(f"Comment char: '{style.comment_char}'")
print(f"Max line length: {style.max_line_length}")

Format: fixed
Comment char: 'c'
Max line length: 72


In [11]:
# Generate F77 code - comments automatically use 'c'
code_f77 = [
    programm('hello', [
        commentf('This is a Fortran 77 program'),
        commentf('Comments use c in column 1'),
        declaref('integer', ['i']),
        dom('i', 1, 10, [
            equalf('x(i)', 'i * 2')
        ])
    ])
]

genfor('output_f77.f', code_f77)

# Display generated code
with open('output_f77.f', 'r') as f:
    print(f.read())

      program hello
c     This is a Fortran 77 program
c     Comments use c in column 1
      integer i
      do 100 i=1, 10
      x(i) = i * 2
  100 continue
      end



In [12]:
# Subroutines also respect the global style
code_sub_f77 = [
    subroutinem('calculate', ['x', 'y', 'result'], [
        commentf('Calculate sum of two numbers'),
        declaref('real*8', ['x', 'y', 'result']),
        equalf('result', 'x + y')
    ])
]

genfor('subroutine_f77.f', code_sub_f77)

with open('subroutine_f77.f', 'r') as f:
    print(f.read())

c
c SUBROUTINE calculate
c
      subroutine calculate(x, y, result)
c     Calculate sum of two numbers
      real*8 x, y, result
      result = x + y
      end subroutine calculate



## 2. Fortran 90 Style (Modern)

Switch to F90 - all subsequent code will use:
- `!` for comments (anywhere in line)
- 132 character line limit
- Free format continuation with `&`

In [13]:
# Switch to F90 style
set_fortran_style('f90')

# Check current style
style = get_fortran_style()
print(f"Format: {style.format}")
print(f"Comment char: '{style.comment_char}'")
print(f"Max line length: {style.max_line_length}")

Format: free
Comment char: '!'
Max line length: 132


In [14]:
# Generate F90 code - same API calls, different output!
code_f90 = [
    programm('hello', [
        commentf('This is a Fortran 90 program'),
        commentf('Comments use ! (modern style)'),
        declaref('integer', ['i']),
        dom('i', 1, 10, [
            equalf('x(i)', 'i * 2')
        ])
    ])
]

genfor('output_f90.f90', code_f90)

with open('output_f90.f90', 'r') as f:
    print(f.read())

program hello
  !    This is a Fortran 90 program
  !    Comments use ! (modern style)
  integer i
  do 100 i=1, 10
    x(i) = i * 2
  100 continue
end



In [15]:
# Subroutines in F90 style
code_sub_f90 = [
    subroutinem('calculate', ['x', 'y', 'result'], [
        commentf('Calculate sum of two numbers'),
        declaref('real*8', ['x', 'y', 'result']),
        equalf('result', 'x + y')
    ])
]

genfor('subroutine_f90.f90', code_sub_f90)

with open('subroutine_f90.f90', 'r') as f:
    print(f.read())

!
! SUBROUTINE calculate
!
subroutine calculate(x, y, result)
  !    Calculate sum of two numbers
  real*8 x, y, result
  result = x + y
end subroutine calculate



## 3. Side-by-Side Comparison

Let's generate the same code in both styles to see the differences:

In [16]:
# Create the same program structure
def create_program():
    return [
        programm('demo', [
            commentf('Program to demonstrate style differences'),
            declaref('integer', ['i', 'sum']),
            equalf('sum', '0'),
            dom('i', 1, 100, [
                commentf('Accumulate sum'),
                equalf('sum', 'sum + i')
            ]),
            commentf('End of program')
        ])
    ]

# Generate F77 version
set_fortran_style('f77')
genfor('demo_f77.f', create_program())

# Generate F90 version
set_fortran_style('f90')
genfor('demo_f90.f90', create_program())

# Display both
print("="*60)
print("FORTRAN 77 VERSION (demo_f77.f)")
print("="*60)
with open('demo_f77.f', 'r') as f:
    print(f.read())

print("\n" + "="*60)
print("FORTRAN 90 VERSION (demo_f90.f90)")
print("="*60)
with open('demo_f90.f90', 'r') as f:
    print(f.read())

FORTRAN 77 VERSION (demo_f77.f)
      program demo
c     Program to demonstrate style differences
      integer i, sum
      sum = 0
      do 100 i=1, 100
c     Accumulate sum
      sum = sum + i
  100 continue
c     End of program
      end


FORTRAN 90 VERSION (demo_f90.f90)
program demo
  !    Program to demonstrate style differences
  integer i, sum
  sum = 0
  do 100 i=1, 100
    !    Accumulate sum
    sum = sum + i
  100 continue
  !    End of program
end

